# Testing Calibrated Angles


In [9]:
# ruff: noqa: E402

%load_ext autoreload
%autoreload 2

# find the root of the project
import os
from pathlib import Path
import sys
import polars as pl

ROOT = Path(os.getcwd()).parent
while not ROOT.joinpath(".git").exists():
    ROOT = ROOT.parent

# add the root to the python path
sys.path.append(str(ROOT))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## October Data


In [10]:
from src.pipelines.open_file import prep_df
from src.radar import CalibratedRadar


radar_obj = CalibratedRadar(
    radar_location_path=ROOT / "configuration" / "october_calibrated.yaml",
)

radar_df = (
    pl.scan_parquet(
        ROOT.joinpath("data", "raw", "*.parquet"),
    )
    .with_columns(
        pl.col("epoch_time").dt.replace_time_zone("UTC"),
    )
    .filter(pl.col("epoch_time").dt.month() == 10)
    .pipe(prep_df, f=radar_obj)
    .pipe(radar_obj.radar_to_latlon)
)

function: create_object_id took: 0.0009028911590576172 seconds
function: filter_short_trajectories took: 0.004970073699951172 seconds
function: clip_trajectory_end took: 0.0018279552459716797 seconds
function: resample took: 0.0010366439819335938 seconds
function: fix_duplicate_positions took: 0.000270843505859375 seconds
function: set_timezone took: 2.7179718017578125e-05 seconds
function: add_cst_timezone took: 2.9087066650390625e-05 seconds
function: add_heading took: 0.00018906593322753906 seconds
function: rotate_radars took: 0.0009639263153076172 seconds
function: update_origin took: 0.0002968311309814453 seconds
function: radar_to_latlon took: 1.4612438678741455 seconds


In [11]:
all_dfs = []
for df in radar_df.select(["ip", "lat", "lon"]).partition_by("ip"):
    all_dfs.append(df.sample(10_000))

df = pl.concat(all_dfs)
df.write_csv("october.csv")

## March Data


In [6]:
from src.pipelines.open_file import prep_df
from src.radar import CalibratedRadar


radar_obj = CalibratedRadar(
    radar_location_path=ROOT / "configuration" / "march_calibrated.yaml",
)

radar_df = (
    pl.scan_parquet(
        ROOT.joinpath("data", "raw", "*.parquet"),
    )
    .with_columns(
        pl.col("epoch_time").dt.replace_time_zone("UTC"),
    )
    .filter(pl.col("epoch_time").dt.month() == 3)
    .collect()
    .lazy()
    .pipe(prep_df, f=radar_obj)
    .pipe(radar_obj.radar_to_latlon)
)

function: create_object_id took: 0.15714097023010254 seconds
function: filter_short_trajectories took: 0.16297006607055664 seconds
function: clip_trajectory_end took: 0.002424001693725586 seconds
function: resample took: 0.02349686622619629 seconds
function: fix_duplicate_positions took: 0.000164031982421875 seconds
function: set_timezone took: 0.002084016799926758 seconds
function: add_cst_timezone took: 7.009506225585938e-05 seconds
function: add_heading took: 6.198883056640625e-05 seconds
function: rotate_radars took: 0.0010559558868408203 seconds
function: update_origin took: 0.0013167858123779297 seconds
function: radar_to_latlon took: 0.7451262474060059 seconds


In [8]:
all_dfs = []
for df in radar_df.select(["ip", "lat", "lon"]).partition_by("ip"):
    all_dfs.append(df.sample(10_000))

df = pl.concat(all_dfs)
df.write_csv("march.csv")